In [1]:
%load_ext autoreload
%autoreload 2

import os
import re
import shutil
import random
import pprint
import itertools
import functools
import collections
import subprocess
import tempfile

import Bio.Seq
import Bio.SeqRecord
import Bio.Align
import Bio.AlignIO
import Bio.Align.AlignInfo
import pysam
import pyranges as pr
import numpy as np
import pandas as pd

import sys
sys.path.append('/home/users/pjh/scripts/python_genome_package_dev/')

In [2]:
from handygenome import common
from handygenome.common import ChromDict, Vcfspec, Interval, IntervalList
from handygenome.variantplus.breakends import Breakends
from handygenome.variantplus.variantplus import VariantPlus, VariantPlusList
from handygenome.variantplus.vcfplus import VcfPlus
from handygenome import igvhandle

[W::hts_idx_load3] The index file is older than the data file: /home/users/pjh/scripts/python_genome_packages/data/popfreq/dbSNP_b155_GRCh37.p13.vcf.gz.csi
[W::hts_idx_load3] The index file is older than the data file: /home/users/pjh/scripts/python_genome_packages/data/popfreq/dbSNP_b155_GRCh37.p13.vcf.gz.csi


In [3]:
FASTA_PATH_HG19 = "/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.fasta"
FASTA_PATH_HG38 = "/home/users/data/01_reference/human_g1k_v38/Homo_sapiens_assembly38.fasta"

FASTA_HG19 = pysam.FastaFile(FASTA_PATH_HG19)
FASTA_HG38 = pysam.FastaFile(FASTA_PATH_HG38)
CHROMDICT_HG19 = ChromDict(fasta_path=FASTA_PATH_HG19)
CHROMDICT_HG38 = ChromDict(fasta_path=FASTA_PATH_HG38)

In [4]:
from handygenome.cnv import sequenza_loader

In [59]:
intvlist = CHROMDICT_HG19.to_interval_list()

In [64]:
intvlist.sort_intervals(CHROMDICT_HG19)

In [65]:
intvlist

[<Interval> 1:1-249,250,621 (1-based),
 <Interval> 2:1-243,199,373 (1-based),
 <Interval> 3:1-198,022,430 (1-based),
 <Interval> 4:1-191,154,276 (1-based),
 <Interval> 5:1-180,915,260 (1-based),
 <Interval> 6:1-171,115,067 (1-based),
 <Interval> 7:1-159,138,663 (1-based),
 <Interval> 8:1-146,364,022 (1-based),
 <Interval> 9:1-141,213,431 (1-based),
 <Interval> 10:1-135,534,747 (1-based),
 <Interval> 11:1-135,006,516 (1-based),
 <Interval> 12:1-133,851,895 (1-based),
 <Interval> 13:1-115,169,878 (1-based),
 <Interval> 14:1-107,349,540 (1-based),
 <Interval> 15:1-102,531,392 (1-based),
 <Interval> 16:1-90,354,753 (1-based),
 <Interval> 17:1-81,195,210 (1-based),
 <Interval> 18:1-78,077,248 (1-based),
 <Interval> 19:1-59,128,983 (1-based),
 <Interval> 20:1-63,025,520 (1-based),
 <Interval> 21:1-48,129,895 (1-based),
 <Interval> 22:1-51,304,566 (1-based),
 <Interval> X:1-155,270,560 (1-based),
 <Interval> Y:1-59,373,566 (1-based),
 <Interval> MT:1-16,569 (1-based),
 <Interval> GL000207.1:1

In [35]:
intvlist_gr = intvlist.to_gr()

In [7]:
intvlist.write_bed('/home/users/pjh/tmp/zzz.bed.gz')

In [67]:
var = 'hihi'
f'{repr(var)}'

"'hihi'"

In [72]:
var = 2
repr(var)

'2'

In [85]:
split_intvlists = intvlist.get_split_interval_lists(num=10)

In [88]:
[(intvlist, intvlist.length) for intvlist in split_intvlists]


[([<Interval> 1:1-249,250,621 (1-based), <Interval> 2:1-60,929,853 (1-based)],
  310180474),
 ([<Interval> 2:60,929,854-243,199,373 (1-based),
   <Interval> 3:1-127,910,954 (1-based)],
  310180474),
 ([<Interval> 3:127,910,955-198,022,430 (1-based),
   <Interval> 4:1-191,154,276 (1-based),
   <Interval> 5:1-48,914,722 (1-based)],
  310180474),
 ([<Interval> 5:48,914,723-180,915,260 (1-based),
   <Interval> 6:1-171,115,067 (1-based),
   <Interval> 7:1-7,064,869 (1-based)],
  310180474),
 ([<Interval> 7:7,064,870-159,138,663 (1-based),
   <Interval> 8:1-146,364,022 (1-based),
   <Interval> 9:1-11,742,658 (1-based)],
  310180474),
 ([<Interval> 9:11,742,659-141,213,431 (1-based),
   <Interval> 10:1-135,534,747 (1-based),
   <Interval> 11:1-45,174,954 (1-based)],
  310180474),
 ([<Interval> 11:45,174,955-135,006,516 (1-based),
   <Interval> 12:1-133,851,895 (1-based),
   <Interval> 13:1-86,497,017 (1-based)],
  310180474),
 ([<Interval> 13:86,497,018-115,169,878 (1-based),
   <Interval> 14

In [16]:
tbam_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/14/LU-14.tumor.bam'
nbam_path = '/home/users/team_projects/Lung_Cancer_Panel_data/03_Data_from_YTKim/02_BAM/14/LU-14.normal.bam'
fasta_path = FASTA_PATH_HG19
outfile_path = '/home/users/pjh/practice/pipeline_test/handygenome/sequenza/test.seqz.gz'
gcwiggle_path = '/home/users/data/01_reference/human_g1k_v37/human_g1k_v37.gc50Base.txt.gz'
chrom = '1'
start0 = 10_000_000
end0 = 11_000_000

handygenome.cnv.sequenza.seqzutils_bam2seqz(tbam_path=tbam_path, nbam_path=nbam_path, 
                                            outfile_path=outfile_path, 
                                            refver='hg19',
                                            chrom=chrom, start0=start0, end0=end0)

/home/users/pjh/practice/pipeline_test/handygenome/sequenza/test.seqz.gz


[mpileup] 1 samples in 1 input files
[mpileup] 1 samples in 1 input files


In [13]:
outfile_path.endswith('.seqz.gz')

True

In [49]:
gr1 = pr.from_dict({'Chromosome': ['1', '2'], 'Start': [0, 100], 'End': [10, 150]})
gr2 = pr.from_dict({'Chromosome': ['1', '2'], 'Start': [5, 200], 'End': [50, 220]})

In [50]:
gr1

,Chromosome,Start,End
0,1,0,10
1,2,100,150


In [51]:
gr2

,Chromosome,Start,End
0,1,5,50
1,2,200,220


In [52]:
gr1.set_union(gr2)

,Chromosome,Start,End
0,1,0,50
1,2,100,150
2,2,200,220


In [58]:
repr('abc')

"'abc'"